In [1]:
import sys
import numpy as np
from math import ceil
import tensorflow as tf
from tensorflow.math import log
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB2
from tensorflow import zeros_like,reduce_sum,divide,exp,expand_dims
from tensorflow.keras.layers import Input, TimeDistributed,Flatten,Dense,Dropout,Reshape


sys.path.append('/kaggle/input/mlware-preprocessed')
from DataHelper import *

In [ ]:
helpMe()

In [ ]:
x_train,x_test = load_images("/kaggle/input/mlware-preprocessed/Faces_train",valid_split=0.2)

In [ ]:

y=PermutationMatrix("/kaggle/input/mlware-preprocessed/Preprocessed_faces.csv")
y_train,y_test=y[:ceil(y.shape[0]*0.8)],y[ceil(y.shape[0]*0.8):]


In [ ]:
newx_train,newy_train=AugmentedData(x_train,y_train,6000,augments_per_image=15)

In [ ]:
newx_test,newy_test=AugmentedData(x_test,y_test,400,augments_per_image=15)

In [2]:
class SinkhornLayer(tf.keras.layers.Layer):
    def __init__(self, epsilon=0.1, n_iters=10):
        super().__init__()
        self.epsilon = epsilon
        self.n_iters = n_iters

    def sinkhorn(self, x):
        # Sinkhorn iterations
        self.u = zeros_like(x[:, 0])
        self.v = zeros_like(x[0])
        for i in range(self.n_iters):
            self.u = -self.epsilon * (log(reduce_sum(exp(divide(x - expand_dims(self.v, axis=1), self.epsilon)), axis=2))) + self.u
            self.v = -self.epsilon * (log(reduce_sum(exp(divide(x - expand_dims(self.u, axis=2), self.epsilon)), axis=1))) + self.v

        # Compute the sinkhorn matrix
        sinkhorn_matrix = exp(divide(x - expand_dims(self.u, axis=2) - expand_dims(self.v, axis=1), self.epsilon))

        return sinkhorn_matrix
def create_siamese_network():
    # Define the input layer
    inputs = Input(shape=(36, 50, 50, 3))
    
    # Define the shared layers of the Siamese network using EffiecientNet architecture
    shared_layers = EfficientNetB2(
        input_shape=(50, 50, 3),
        include_top=False,
        weights="imagenet"
    )
    shared_layers.trainable = False

    # Apply the shared layers to each input image
    x = TimeDistributed(shared_layers)(inputs)
    x = Flatten()(x)

    # Add a fully connected layer to output a vector of length 1296
    fc1 = Dense(1024, activation='relu')(x)
    fc1 = Dropout(0.2)(fc1)  # Add dropout
    fc2 = Dense(512, activation='relu')(fc1)
    fc2 = Dropout(0.2)(fc2)  # Add dropout
    output = Dense(1296, activation='softmax')(fc2)
    
    # Reshape output to a 36x36 one-hot encoded matrix
    reshaped_output = Reshape((36, 36))(output)
    sinkhorn_output = SinkhornLayer()(reshaped_output)
    
    # Define the Siamese model with 36 inputs and one output
    model = Model(inputs=inputs, outputs= sinkhorn_output)
    

    return model

# Create the Siamese network with input shape (50, 50, 3) and outputting a 36x36 one-hot encoded matrix
siamese_network = create_siamese_network()
siamese_network.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.00001),metrics=["accuracy"])


31790344/31790344 [==============================] - 0s 0us/step


In [3]:
siamese_network.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 36, 50, 50, 3)]   0         
                                                                 
 time_distributed (TimeDistr  (None, 36, 2, 2, 1408)   7768569   
 ibuted)                                                         
                                                                 
 flatten (Flatten)           (None, 202752)            0         
                                                                 
 dense (Dense)               (None, 1024)              207619072 
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                             

In [ ]:
siamese_network.fit(newx_train,newy_train,validation_data=(newx_test,newy_test),epochs=500)

In [ ]:
siamese_network.save_weights("model_faces.h5")

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Flatten, TimeDistributed, Softmax

# Define the CNN
cnn = Sequential()
cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Conv2D(128, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Conv2D(128, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Flatten())

# Define the LSTM
lstm = Sequential()
lstm.add(LSTM(128, input_shape=(36, 36)))

# Add a TimeDistributed layer with a Dense layer with Softmax activation function
time_distributed_layer = Sequential()
time_distributed_layer.add(TimeDistributed(Dense(36, activation='softmax')))

# Combine the CNN, LSTM, and TimeDistributed layers
model = Sequential()
model.add(TimeDistributed(cnn, input_shape=(36, 50, 50, 3)))
model.add(TimeDistributed(Flatten()))
model.add(lstm)
model.add(time_distributed_layer)

# Define a function to add up the outputs of the TimeDistributed layer into a matrix
def sum_vectors(x):
    return K.sum(x, axis=1)

# Add a layer to sum the outputs of the TimeDistributed layer into a matrix
model.add(TimeDistributed(Dense(36, activation='softmax')))
model.add(Flatten())
model.add(Dense(36*36, activation='linear'))
model.add(Reshape((36, 36)))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')


ValueError: Exception encountered when calling layer "sequential_52" (type Sequential).

Input 0 of layer "lstm_12" is incompatible with the layer: expected shape=(None, None, 36), found shape=(None, 36, 128)

Call arguments received by layer "sequential_52" (type Sequential):
  • inputs=tf.Tensor(shape=(None, 36, 128), dtype=float32)
  • training=None
  • mask=None

In [15]:
model.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_17 (TimeDi  (None, 36, 128)          240832    
 stributed)                                                      
                                                                 
 time_distributed_18 (TimeDi  (None, 36, 128)          0         
 stributed)                                                      
                                                                 
 sequential_27 (Sequential)  (None, 128)               131584    
                                                                 
Total params: 372,416
Trainable params: 372,416
Non-trainable params: 0
_________________________________________________________________
